<a href="https://colab.research.google.com/github/OscarRojasG/CLP-Framework/blob/main/CLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generador de instancias

Cada instancia se representa con el siguiente formato:

```
1                   # ID de la instancia
587 233 220         # Dimensiones del contenedor (L W H)
10                  # Número de tipos de cajas
1 108 0 76 0 30 1 20
2 110 0 43 1 25 1 11
3 92 1 81 1 55 1 9
4 81 0 33 1 28 1 9
5 120 1 99 1 73 1 9
6 111 0 70 1 48 1 13
7 98 0 72 1 46 1 4
8 95 0 66 0 31 1 12
9 85 0 84 0 30 1 9
10 71 0 32 1 25 1 10
```

Cada tipo de caja se describe en una línea con el siguiente formato:

```
id_caja  L  rotX  W  rotY  H  rotZ  cantidad
1 108 0 76 0 30 1 20
```

Donde:

* $L, W, H$ son las dimensiones de la caja
* $rotX, rotY, rotZ$ es la factibilidad de que cada caja pueda ser rotada sobre los ejes X, Y, Z.

In [2]:
from typing import List
import random

class BoxType:
    def __init__(self, box_id, dims, rots, qty):
        self.id = box_id        # identificador de la caja
        self.dims = dims        # (L, W, H)
        self.rots = rots        # (rotX, rotY, rotZ)
        self.qty = qty          # cantidad de cajas


class CLPInstance:
    def __init__(self, container, boxes : list[BoxType]):
        self.container = container  # (L, W, H)
        self.n_types = len(boxes)
        self.boxes = boxes          # lista de BoxType


def instance(n_types=10, seed=None):
    """
    Genera una sola instancia del CLP como objeto CLPInstance.
    """
    if seed is not None:
        random.seed(seed)

    # Dimensiones del contenedor (fijas en este caso)
    l, w, h = 587, 233, 220

    # Límites de dimensiones de las cajas
    alpha = [30, 25, 20]   # mínimo largo, ancho, alto
    beta  = [120, 100, 80] # máximo largo, ancho, alto
    L = 2  # constante de estabilidad

    # Volumen del contenedor
    tc = l * w * h

    dimension_box = []
    cantidad_box_type = []
    volumen_box_type = []
    orientacion_box = []

    # Genera tipos de cajas
    for i in range(n_types):
        # Dimensiones aleatorias dentro de los rangos
        r_j = [random.randint(alpha[j], beta[j]) for j in range(3)]
        aux_dim = [alpha[j] + (r_j[j] % (beta[j] - alpha[j] + 1)) for j in range(3)]
        dimension_box.append(aux_dim)

        # Inicializa cantidad
        cantidad_box_type.append(1)

        # Volumen de la caja
        volumen_box_type.append(aux_dim[0] * aux_dim[1] * aux_dim[2])

        # Orientación factible según la constante L
        min_dim = min(aux_dim)
        aux_orient = [1 if aux_dim[j] / min_dim < L else 0 for j in range(3)]
        orientacion_box.append(aux_orient)

    # Rellena hasta que no quepa más
    volumen_cargo = 0
    while True:
        volumen_cargo = sum(cantidad_box_type[i] * volumen_box_type[i] for i in range(n_types))
        aux = random.randint(0, n_types - 1)
        v_k = volumen_box_type[aux]
        if tc > volumen_cargo + v_k:
            cantidad_box_type[aux] += 1
        else:
            break

    # Construye lista de BoxType
    boxes = [
        BoxType(i + 1, dimension_box[i], orientacion_box[i], cantidad_box_type[i])
        for i in range(n_types)
    ]

    return CLPInstance(container=(l, w, h), boxes=boxes)


def gen_instances(filename, n_instances=100, n_types=10, seed=None):
    """
    Genera un archivo con varias instancias CLP en el formato definido.
    """
    with open(filename, "w") as f:
        # número total de instancias
        f.write(str(n_instances) + "\n")

        for inst_id in range(1, n_instances + 1):
            inst = instance(n_types=n_types, seed=(seed + inst_id) if seed is not None else None)

            # encabezado de la instancia
            f.write(f"{inst_id}\n")
            l, w, h = inst.container
            f.write(f"{l} {w} {h}\n")
            f.write(f"{inst.n_types}\n")

            # tipos de cajas
            for i, box in enumerate(inst.boxes):
                Lc, Wc, Hc = box.dims
                rotX, rotY, rotZ = box.rots
                qty = box.qty

                if inst_id == n_instances and i == len(inst.boxes) - 1:
                    f.write(f"{box.id} {Lc} {rotX} {Wc} {rotY} {Hc} {rotZ} {qty}")
                else:
                    f.write(f"{box.id} {Lc} {rotX} {Wc} {rotY} {Hc} {rotZ} {qty}\n")

def read_instances(filename) -> list[CLPInstance]:
    """
    Lee un archivo de instancias CLP y devuelve una lista de objetos CLPInstance.
    """
    instances = []
    with open(filename, "r") as f:
        # número total de instancias
        n_instances = int(f.readline().strip())

        for _ in range(n_instances):
            inst_id = int(f.readline().split()[0])  # id de la instancia

            # dimensiones del contenedor
            l, w, h = map(int, f.readline().split())

            # número de tipos de cajas
            n_types = int(f.readline().strip())

            boxes = []
            for _ in range(n_types):
                parts = f.readline().split()
                box_id = int(parts[0])
                Lc = int(parts[1])
                rotX = int(parts[2])
                Wc = int(parts[3])
                rotY = int(parts[4])
                Hc = int(parts[5])
                rotZ = int(parts[6])
                qty = int(parts[7])

                boxes.append(BoxType(box_id, (Lc, Wc, Hc), (rotX, rotY, rotZ), qty))

            instances.append(CLPInstance(container=(l, w, h), boxes=boxes))

    return instances

In [3]:
gen_instances("prueba.txt", n_instances=5, n_types=10, seed=1)
read_instances("prueba.txt")

# BSG-VCS & Generación de datos

Al ejecutar

```
./BSG_CLP prueba.txt -i 1 -t 10 --verbose2=5 > output.txt
```

se guardarán los bloques seleccionados por BSG para construir la solución, en conjunto con los $m=5$ bloques más prometedores para cada estado:

```
selected block:686 space:(362,0,103)
  action block:686 eval: 752760 0.024314749 0.85935438 0.16666667 279914.38
  action block:600 eval: 752760 0.1954876 0.76418638 0.16666667 144329.86
  action block:147 eval: 376380 0.024314749 0.79643276 0.33333333 118606.5
  action block:149 eval: 501840 0.024314749 0.73896542 0.25 110652.05
  action block:123 eval: 501840 0.11661384 0.74613438 0.25 104129.09
```

Los valores a continuación de `eval` indican las métricas:

```
eval: V Loss CS 1/n VCS
```

Nótese que en algunos casos, el bloque elegido por BSG no está dentro de los $m$ más prometedores. Estos casos son de particular relevancia ya que la heurística "falla" en su evaluación. En estos casos se imprime un sexto bloque:

```
selected block:141 space:(282,169,113)
  action block:2318 eval: 825600 0.068243858 0.68818263 0.2 125052.61
  action block:550 eval: 495360 0.041233657 0.66613802 0.33333333 75069.99
  action block:553 eval: 660480 0.055983909 0.60071472 0.25 61531.879
  action block:568 eval: 660480 0.26440305 0.61178143 0.25 51578.982
  action block:17 eval: 194532 0.007518797 0.70680679 1 48185.48
  action block:141 eval: 330240 0.01910828 0.62544895 0.5 43153.121
```


In [16]:
%cd /content
!chmod +x BSG_CLP
!./BSG_CLP prueba.txt -i 1 -t 10 --verbose2=5 > output.txt

/content
